<a href="https://colab.research.google.com/github/WinC3/SDSS-Datathon-2025/blob/main/XGBoostTreeModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install torch torchvision torchaudio pytorch-tabnet scikit-learn pandas numpy

ERROR: Operation cancelled by user


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from pytorch_tabnet.tab_model import TabNetRegressor
from geopy.distance import geodesic

In [186]:
from google.colab import files
uploaded = files.upload()

Saving pruned-estate-data.csv to pruned-estate-data.csv


In [243]:
# Load dataset (Upload or use Google Drive)
df = pd.read_csv("xgboost-data.csv")  # Replace with actual dataset

# Drop columns that contain 'N/A' (NaN) in any row
df.dropna(axis=1, how='any', inplace=True)

# Drop ID column
df.drop(columns=["id_"], inplace=True, errors='ignore')

# Convert "DEN" from "Yes"/"No" to 1/0
if "DEN" in df.columns:
    df["DEN"] = df["DEN"].map({"YES": 1, "No": 0})

# Convert "size" from "0-499 sqft" format to separate min and max columns
def extract_size_range(size_str):
    if isinstance(size_str, str) and "-" in size_str:
        size_range = size_str.split(" sqft")[0].split("-")  # Remove "sqft" and split range
        return int(size_range[0]), int(size_range[1])
    return np.nan, np.nan  # Handle unexpected formats

if "size" in df.columns:
    df["size_min"], df["size_max"] = zip(*df["size"].apply(extract_size_range))
    df.drop(columns=["size"], inplace=True)  # Drop original 'size' column

# Drop rows where size couldn't be converted (if any)
df.dropna(subset=["size_min", "size_max"], inplace=True)

# One-hot encode categorical features (exposure & ward)
if "exposure" in df.columns:
    df = pd.get_dummies(df, columns=["exposure"])
if "ward" in df.columns:
    df = pd.get_dummies(df, columns=["ward"])
if "parking" in df.columns:
    df = pd.get_dummies(df, columns=["parking"])

# Define features and target
X = df.drop(columns=["price"]).values  # Features
y = df["price"].values  # Target
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle Missing Values in X_train and X_test
imputer = SimpleImputer(strategy="median")  # Use median for robustness
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Normalize y_train and y_test (TARGET VARIABLE - PRICE)
y_mean = y_train.mean()
y_std = y_train.std()
y_train = (y_train - y_mean) / y_std  # Normalize target variable
y_test = (y_test - y_mean) / y_std  # Use same mean & std from training set


In [244]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [245]:
from xgboost import XGBRegressor
# Train XGBoost model
model = XGBRegressor(
    n_estimators=500,  # Number of trees (increase if needed)
    learning_rate=0.05,  # Step size (reduce if unstable)
    max_depth=3,  # Depth of each tree (increase if underfitting)
    subsample=0.8,  # Use 80% of data per tree (reduces overfitting)
    colsample_bytree=0.8,  # Use 80% of features per tree
    random_state=42
)
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Convert predictions back to original price scale
y_pred_original = (y_pred * y_std) + y_mean
y_test_original = (y_test * y_std) + y_mean

# Calculate MAE
mae = mean_absolute_error(y_test_original, y_pred_original)
print(f"XGBoost Test MAE: ${mae:,.2f}")

mape = np.mean(np.abs((y_test_original - y_pred_original) / y_test_original)) * 100
print(f"XGBoost Test MAPE: {mape:.2f}%")

XGBoost Test MAE: $107,633.31
XGBoost Test MAPE: 11.59%
